# EZyRB

# Tutorial 1: Model Reduction on a vtk pressure field - offline phase

In this tutorial we will show the typical workflow for the construcion of the reduced basis space. In this phase, called <b>Offline</b>, the high-fidelity solutions (*snapshots*) are imported from files and they are combined to create the space.<br>
First of all we just import the required packages.

In [1]:
from ezyrb.offline import Offline
from ezyrb.podinterpolation import PODInterpolation

import numpy as np

Then, we create the new instance; the class constructor parameters are:
- *output_name*: the name of the variable (output) we want to extract from the solution files.
- *space_type*: the method we want to use to generate the reduced basis space; default value is <tt>Pod</tt>.
- *weight_name*: the (optional) name of the variable (output) we want to extract from a solution file and use to obtain the weighted snapshots; default value is None.
- *dformat*: the flag which allows to chose if use the solution stored by cells or by points; allowed values are 'cell' or 'point'; default value is 'cell'.

In [2]:
offline_phase = Offline(
    output_name = 'Pressure',
    space_type  = PODInterpolation,
    weight_name = None,
    dformat     = 'point'
)

## Import Solution Files

Now, we need to select the files which contain the previously computed solutions of the parametric problem. In this case, we want to approximate the pressure field; typically, the first solutions to add at the *database* are the solutions computed at the corners of the parametric domain. Because, in this example, we are using a 2D parameters, 4 files are required.
You can view the pressure field (computed with the high-fidelity method) corresponding to these 4 corners of the parametric domain in the following pictures.

![](pictures/pressure_in_corners.png)

We can import the files in two ways:

In [3]:
mu_values = [
    [-.5, -.5],
    [ .5, -.5],
    [ .5,  .5],
    [-.5,  .5]
]
files = [
    "../tests/test_datasets/matlab_00.vtk",
    "../tests/test_datasets/matlab_01.vtk",
    "../tests/test_datasets/matlab_02.vtk",
    "../tests/test_datasets/matlab_03.vtk"
]
offline_phase.init_database(mu_values, files)

In this case, we manualy construct a list that contains the parametric points and a list that contains the name of the solution files; obviously, the *i*-th file has to contain the high-fidelity solution computed at the *i*-th parametric point (in this example, "tests/test_datasets/matlab_02.vtk" contains the solution of the problem with the parameters equal to {0.5, 0.5}).


Alternatively, we can indicate the parametric points and the files by writing them on a text file and passing this file as argument of `init_database_from_file` method. Below, we display the file for our example.

In [4]:
!cat ../tests/test_datasets/mu.conf

# ================================================= # 
# Mu file                                           #
# ================================================= # 

# Lines starting with '#' are comments, they will be ignored.

# mu0   mu1     file
-.5    -.5      tests/test_datasets/matlab_00.vtk
 .5    -.5      tests/test_datasets/matlab_01.vtk
 .5     .5      tests/test_datasets/matlab_02.vtk
-.5     .5      tests/test_datasets/matlab_03.vtk


## Error Approximation & Improvement

At the moment, the database is composed by 4 files. Before proceding with the reduced basis space generation, we would have an idea of the approximation accuracy we are able to reach with these high-fidelity solutions. The library provides an efficient *a priori* error estimator.

In [5]:
offline_phase.loo_error(func=np.linalg.norm)

array([ 0.1491303 ,  0.05875266,  0.04603029,  0.07641878])

Using the *leave-one-out* strategy, an error is computed for each parametric point in our database and these values are returned as array. Moreover, we can use the information about the errors to locate the parametric points where we have to compute the new high-fidelity solutions and add these to the database in order to optimally improve the accuracy.

In [6]:
offline_phase.optimal_mu(
    error = offline_phase.loo_error(func=np.linalg.norm),
    k = 2
)

[array([ 0.07826684,  0.17576105]), array([-0.293344  , -0.23120537])]

These function can be used to achieve the wanted (estimated) accuracy. An iterative example is:

In [7]:
error = offline_phase.loo_error()
toll = 0.07
while max(error) > toll:
    
    new_points = offline_phase.optimal_mu(error, k=1)
    print('Current estimated error: {}'.format(max(error)))
    print('Optimal parametric points:')
    
    for point in new_points:
        print('\t{}'.format(point))
        
    for point in new_points:
        # Compute the high-fidelity solution now...
        filename = input('Solution file for point {}: '.format(point))
        offline_phase.add_snapshot(point, filename)
    
    error = offline_phase.loo_error()

Current estimated error: 0.149130299687
Optimal parametric points:
	[-0.293344   -0.23120537]
Solution file for point [-0.293344   -0.23120537]: "../tests/test_datasets/matlab_04.vtk"


For the sake of clarity, we show also the pressure field corresponding to the parametric point [-0.29334401 -0.23120535]

![](pictures/field_04.png)

## Reduced Basis Space Generation and Save

After importing the snapshots, we combine them to build the reduced basis space and save it to a file.

In [8]:
offline_phase.generate_rb_space()
offline_phase.save_rb_space('pod_space')

The offline phase is concluded: the reduced basis space will be loaded during the online phase to obtain the approximation in a new parametric point.